In [2]:
from IPython.display import Image

> representation learning & (latent) generation

https://www.youtube.com/watch?v=Dg0YcABQ_aU

- AE 是 determinstic，VAE 有一个 random 的过程
- VAE：latent code distribution constraints
    - the compact representation follows a distribution
    - 建模的是joint distribution (over data & latent variables)
        - $p(x,z)=p(z)\cdot p_\theta(x|z)$
        - learning objective: maximize the corpus log likelihood
$$
\log P(\mathcal X)=\sum_{x\in\mathcal X}\log P(x;\theta)
$$
- $P(x;\theta)=\int P(x|z;\theta)P(z)dz$: the marginal likelihood of a single datapoint $x$
    - $P(x;\theta)\approx \sum_{z\sim P(z)}P(x|z;\theta)$
- two tasks
    - learn to generate data from the latent code: $p_\theta(x|z)$
    - learn the distribution of latent codes: $p_\theta(z|x)$
        - $p(z|x)=\frac{p(z)(x|z)}{p(x)}=\frac{p(x|z)p(z)}{\int p(z)p(x|z)dz}$（分母部分是 intractable 的）
    - variational inference approximates the true posterior $p_\theta(z|x)$ with a family of distribution $q_\phi(z|x)$
        - $\min KL(q_\phi(z|x)\|p_\theta(z|x))$
- two tasks
    - Generator (Decoder): $p_\theta(x|z)$
    - Inference (Encoder): $q_\phi(z|x)$

In [4]:
Image(url='./figs/two-tasks-vae.png', width=300)

> log p(x) = ELBO + KL

- **ELBO**: $\int q(z)\log\frac{p(x,z)}{q(z)}dz$
    - $\int q(z|x)\log \frac{p(x,z)}{q(z|x)}dz=\int q(z|x)[\log p(x|z)+\log p(z)-\log q(z|x)]dz$
    - $\int q(z|x)\log p(x|z)dz- \left( \int q(z|x) \log q(z|x) dz - \int q(z|x) \log p(z) dz \right)$
    - $\mathbb{E}_{q(z|x)}[\log p(x|z)]- D_{\text{KL}}(q(z|x) \parallel p(z))$

$$
p(x)=\int p(z)p(x|z)dz.
$$

$$
\log p(x) \geq \mathbb{E}_{q(z|x)}[\log p(x|z)] - D_{\text{KL}}(q(z|x)||p(z))
$$

- 证据下界 (Evidence Lower Bound, ELBO)
    - 既然无法直接最大化 log p(x)，那我们就去最大化它的下界。下界被抬得越高，log p(x) 的真实值也就会越高。
    - $q(z|x)$: 近似后验分布 (Approximate Posterior) 或 编码器 (Encoder)。
    - $\mathbb{E}_{q(z|x)}[...]$: 期望。意思是“从鉴赏家 q(z|x) 推断出的各种可能的‘想法’中，我们进行采样，然后计算一个平均值”。
- training
    - 第一项: $\mathbb{E}_{q(z|x)}[\log p(x|z)]$ (重构项 Reconstruction Term)
    - 第二项: $D_{\text{KL}}(q(z|x)||p(z))$ (正则化项 Regularization Term)
        - 这一项衡量了“鉴赏家” $q(z|x)$ 推断出的“想法”的分布，与我们预设的“灵感库” $p(z)$ 的分布有多接近。我们希望这个KL散度越小越好（即在它前面有个负号，所以最大化整个ELBO等于最小化KL散度）。
- inference
    - first infer：$q(z|x)$
        - 深度学习训练要做的事情，训练 $q(z|x)$ (编码器) 的全部意义，就是为了训练出一个强大的 p(x|z) (解码器)。
    - generate new data/decoding
        - $z \sim p(z) \xrightarrow{\text{解码器 } p(x|z)} x'$
        - 生成新数据时，我们确实不再使用编码器了。
            - 如果没有编码器在训练阶段的参与，解码器将完全不知道如何将一个随机向量 z 变成一张有意义的图片。

### 变分

- https://www.youtube.com/watch?v=u4BJdBCDR9w

- variational inference
    - approximate the posterior ($p(z|x)$) with another distribution $q_\phi(z|x)$
- 用一个简单的、我们已知的概率分布（比如高斯分布 $q(z|x)$），去“近似”一个我们想知道但太复杂、算不出来的真实分布（后验分布 $p(z|x)$)。
    - “变分”的过程，就是寻找最优的 μ 和 σ，让我们定义的这个简单高斯分布 q(z|x)，尽可能地接近那个无法计算的、真实的 p(z|x)。
- 变分贝叶斯
    - 贝叶斯：$p(z|x) = \frac{p(x|z)\cdot p(z)}{p(x)}$
        - $p(x)$: intractable，从 $p(x)=\int p(x,z)dz=\int p(x|z)p(z)dz$ 的角度
    - 一个简单的、可控的 $q(z|x)$，然后通过训练（最小化KL散度，等价于最大化ELBO），让 q(z|x) 的参数（由编码器决定）不断调整，直到它成为 p(z|x) 的一个足够好的“高仿品”。
- 从优化的角度：KL div 作为目标
    - $p_\theta(z|x)\approx q_\phi(z|x)$
        - $p_\theta(z|x)$ 是 ground truth
        - $q_\phi(z|x)$ learn the parameters using optimization
    - $D_{kl}(q_\phi(z|x)\|p_\theta(z|x))$
        - $D_{kl}(q_\phi(z|x)\|p_\theta(z|x))=E_{q_\phi(z|x)}\log\frac{q_\phi(z|x)}{p_\theta(z,x)}+\log p_\theta(x)$ 

### training details

- 对于输入 $x$，VAE 的编码器（Encoder）输出的不是一个确定的潜在向量 $z$，而是潜在空间中一个概率分布的参数。
    - 标准自编码器 (AE)：Encoder(x) → z (一个确定的点/向量)
    - 变分自编码器 (VAE)：Encoder(x) → (μ, σ) (一个高斯分布的均值和标准差)
        - mean vector: $\mu$
        - 对数方差向量 (Log-variance vector): $\log(\sigma^2)$
            - 标准差 $σ$ 必须是正数。如果网络直接输出 $σ$ ，我们需要加一个激活函数（比如ReLU）来保证其非负，这有时会造成梯度问题（比如梯度消失）。
            - 方差 $\sigma^2$ 也必须是正数。
            - 而 $\log⁡(σ^2)$ 的取值范围可以是整个实数域 (−∞,+∞)。网络可以直接输出任何值，然后我们通过取指数 exp 来得到方差 $σ^2$，这样自然就保证了方差是正数，并且在数值上更稳定。
- 从分布中采样得到 $z$
    - 有了这个由 $(\mu,\sigma)$ 定义的分布后，我们并不能直接把这个分布本身送入解码器。解码器需要的是一个具体的向量 $z$
    - 下一步就是从这个分布 $\mathcal N(\mu,\sigma^2)$ 中采样一个点，得到具体的潜在向量 $z$
        - $z\sim \mathcal N(\mu,\sigma^2)$
- “重参数技巧” (Reparameterization Trick)
    - 采样是一个随机过程，它是不可微分的。如果直接从 $\mathcal N(\mu,\sigma^2)$中随机抽一个数，梯度就无法从 $z$ 反向传播到编码器输出的 $\mu$ 和 $\sigma$x 了，为了解决这个问题，VAE 引入了绝妙的“重参数技巧”。它把采样过程重写为：
$$
z = \mu + \sigma \odot \epsilon
$$